In [1]:
#using PyPlot
include("lib/all.jl");

In [2]:
N=2^25
ax = zeros(N)
ay = zeros(N)
az = zeros(N);

particles = make_particles(N);
t = DTree(particles, 12);

In [3]:
@time fill!(t.tree.nodes, Node());

  0.176367 seconds (3.99 k allocations: 175.127 KB)


In [4]:
@time fill!(t.tree.exps, NodeExp());

In [5]:
@time group!(t);

In [6]:
@time inform!(t);

In [7]:
fill!(ax,0.0)
fill!(ay,0.0)
fill!(az,0.0);

In [8]:
@time interact!(t, 0.3, ax, ay, az, 1.0e-3);

In [9]:
@time collect!(t)

In [10]:
@time accel!(t, ax,ay,az)

  0.348384 seconds (72.31 k allocations: 2.335 MB)


In [11]:
mean(abs(ax))/1e5

165.3596835059119

In [12]:
mx,myd,e50,e90,e95,e99,m,s = perf(particles, ax,ay,az, 1000, 1.0e-3)
print(myd)

mean(abs(rax)) = 1.6463081844097648e7
[0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,6.0,11.0,28.0,62.0,84.0,177.0,188.0,169.0,142.0,67.0,41.0,7.0,9.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0]

In [13]:
semilogx(10.^mx,myd,"-k")

LoadError: UndefVarError: semilogx not defined

In [14]:
e50

0.5319718113053477

In [21]:
e90

103.32044517667262

In [22]:
e99

158.85831773539144

In [15]:
function doall(t, ax,ay,az, alpha, eps2)
    total_mass = t.trees[1].total_mass
    for it in t.trees
        fill!(it.exps, NodeExp());
        it.total_mass = total_mass
    end
    tic()
    group!(t)
    t_group = toq()
    tic()
    inform!(t)
    t_inf = toq()
    fill!(ax,0.0)
    fill!(ay,0.0)
    fill!(az,0.0)
    tic()
    interact!(t, alpha, ax, ay, az, eps2)
    t_int = toq()
    tic()
    collect!(t)
    t_coll = toq()
    tic()
    accel!(t,ax,ay,az)
    t_acc = toq()
    t_group, t_inf, t_int, t_coll, t_acc
end

doall (generic function with 1 method)

In [16]:
s_g = Float64[]
s_inf = Float64[]
s_int = Float64[]
s_coll = Float64[]
s_acc = Float64[]
s_all = Float64[]
for i in 1:20
    tic()
    t_group, t_inf, t_int, t_coll, t_acc = doall(t, ax, ay, az, 0.3, 1.0e-3);
    t_all = toq()
    append!(s_g, t_group)
    append!(s_inf, t_inf)
    append!(s_int, t_int)
    append!(s_coll, t_coll)
    append!(s_acc, t_acc)
    append!(s_all, t_all)
    @show i
end

i = 1
i = 2
i = 3
i = 4
i = 5
i = 6
i = 7
i = 8
i = 9
i = 10
i = 11
i = 12
i = 13
i = 14
i = 15
i = 16
i = 17
i = 18
i = 19
i = 

In [17]:
@show mean(s_g)
@show mean(s_inf)
@show mean(s_int)
@show mean(s_coll)
@show mean(s_acc)
@show mean(s_all);

mean(s_g) = 1.3914781284500002
mean(s_inf) = 0.68141024265
mean(s_int) = 1.5525602874
mean(s_coll) = 0.2728066133
mean(s_acc) = 0.21856631989999994
mean(s_all) = 4.3210328338


In [16]:
@show mean(s_g)
@show mean(s_inf)
@show mean(s_int)
@show mean(s_coll)
@show mean(s_acc)
@show mean(s_all);

mean(s_g) = 1.3853150495000002
mean(s_inf) = 0.7206255572
mean(s_int) = 1.8021083284
mean(s_coll) = 2.9465398244
mean(s_acc) = 0.24285650790000002
mean(s_all) = 7.644699567650001


In [13]:
t.trees[1].num_nodes_used / length(t.trees[1].nodes)

0.730268426773536

In [29]:
sizeof(t.tree.nodes)/2^30

1.4025000557303429

In [35]:
length(t.tree.particles)*sizeof(t.tree.particles[1])/2^30

1.0

In [36]:
sizeof(t.tree.exps)/2^30

1.5675000622868538

In [37]:
sizeof(ax)*4*2/2^30

2.0

In [39]:
1.4+1.0+1.57+2

5.97

In [40]:
64/5.97*2^25/2^20

343.0485762144054